In [ ]:
import sagemaker
import boto3

sm_session = sagemaker.Session()
role = sagemaker.get_execution_role()
region = sm_session.boto_region_name
account = sm_session.account_id()


container_name = "deepspeed-sm"
container_tag = "latest"
image = '{}.dkr.ecr.{}.amazonaws.com/{}:{}'.format(account, region, container_name, container_tag)

In [ ]:
!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin 763104351884.dkr.ecr.{region}.amazonaws.com
# loging to your private ECR
!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account}.dkr.ecr.{region}.amazonaws.com

In [ ]:
! ./build_and_push.sh $container_name $container_tag Dockerfile

In [ ]:
! docker images ls

In [ ]:
! pip install 'sagemaker[local]' --upgrade

In [ ]:
from sagemaker.local import LocalSession

# Configure our local training session
sagemaker_local_session = LocalSession()
sagemaker_local_session.config = {'local': {'local_code': True}}

In [ ]:
est = sagemaker.estimator.Estimator(image,
                                    role=role,
                                    instance_count=1,
                                    instance_type='local_gpu',
                                    sagemaker_session=sagemaker_local_session
)

est.fit()